In [1]:
%load_ext nb_black

In [ ]:
from clarifai.rest import ClarifaiApp
API_KEY = "ebd06cff13e44f708b13a9df29c6591b"
MODEL_ID = "d16f390eb32cad478c7ae150069bd2c6"

In [2]:
app = ClarifaiApp(api_key=API_KEY)

In [3]:
model = app.models.get(model_id=MODEL_ID)

In [31]:
response = model.predict_by_filename('coffee.jpg')

In [32]:
response

{'status': {'code': 10000,
  'description': 'Ok',
  'req_id': '552ec583b3b44185a6bc0d3aaac48dbc'},
 'outputs': [{'id': 'b57b0b5f4c194405b86d72858275c88c',
   'status': {'code': 10000, 'description': 'Ok'},
   'created_at': '2019-12-03T13:34:45.798681131Z',
   'model': {'id': 'd16f390eb32cad478c7ae150069bd2c6',
    'name': 'moderation',
    'created_at': '2017-05-16T19:20:38.733764Z',
    'app_id': 'main',
    'output_info': {'message': 'Show output_info with: GET /models/{model_id}/output_info',
     'type': 'concept',
     'type_ext': 'concept'},
    'model_version': {'id': 'aa8be956dbaa4b7a858826a84253cab9',
     'created_at': '2017-10-31T16:30:31.226185Z',
     'status': {'code': 21100, 'description': 'Model is trained and ready'}},
    'display_name': 'Moderation'},
   'input': {'id': '1b12ee475407442a9a218a1329cfcce4',
    'data': {'image': {'url': 'https://s3.amazonaws.com/clarifai-api/img3/prod/orig/f54a0cf1bdea45d1861277af0284558e/35fae8e2aef95e7560383ef8afd9ae39',
      'base6

In [4]:
IMAGE_PATH = "coffee.jpg"

In [4]:
import cv2
im = cv2.imread(IMAGE_PATH)

In [11]:
cv2.imshow('ImageWindow', current_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
from clarifai.rest import ClarifaiApp
API_KEY = "ebd06cff13e44f708b13a9df29c6591b"
MODEL_ID = "d16f390eb32cad478c7ae150069bd2c6"
app = ClarifaiApp(api_key=API_KEY)
model = app.models.get(model_id=MODEL_ID)

In [11]:
import cv2
import numpy as np
IMAGE_PATH = "coffee.jpg"
im = cv2.imread(IMAGE_PATH)
im2 = cv2.resize(im,(50,50))
im3 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
current_image = im3.astype(np.float32) / 255

In [15]:
def image_to_predictions(image_path):
    """Uses the clarifai model to give a prediction vector for image."""
    response = model.predict_by_filename(image_path)
    concepts = response['outputs'][0]['data']['concepts']
    scores_dict = {}
    for concept in concepts:
        scores_dict[concept['name']] = concept['value']
    final_dict = [scores_dict['safe'], scores_dict['suggestive'],scores_dict['explicit'],scores_dict['drug'],scores_dict['gore']]
    return np.asarray(final_dict)

In [16]:
def save_image(image, path):
    """Saves the image to specified path."""
    cv2.imwrite(path, image)

In [17]:
def plus_h_image(current_image, coordinate):
    """Perturbs the image at the specified pixel."""
    plus_image = current_image.copy()
    plus_image[coordinate_numbering_rev[coordinate]] = plus_image[coordinate_numbering_rev[coordinate]]+h
    return plus_image

In [18]:
def pick_random_coordinate():
    """Selects a pixel at random."""
    return np.random.choice(coordinates, 1)[0]

In [19]:
def is_better(goal_coordinate, delta):
    """Goal is to minimize the value of goal_coordinate."""
    return delta[goal_coordinate] < 0

In [20]:
coordinate_numbering = {}
coordinate_numbering_rev = {}
index = 0
for i in range(current_image.shape[1]):
    for j in range(current_image.shape[0]):
            coordinate_numbering[tuple((i,j))]=index
            coordinate_numbering_rev[index]=tuple((i,j))
            index+=1
coordinates = list(coordinate_numbering.values())

In [21]:
GOAL_COORDINATE = 0
NUM_ITERS = 10
LEARNING_RATE = 0.01
h = 0.005
for i in range(NUM_ITERS):
    image_save_path = "current_image"+str(i)+".jpg"
    save_image(current_image*255, image_save_path)
    current_score = image_to_predictions(image_save_path)
    print("Iteration "+str(i)+" score:")
    print(current_score)
    chosen_coordinate = pick_random_coordinate()
    plus_image = plus_h_image(current_image, chosen_coordinate)
    save_image(plus_image*255,"plus_image.jpg")
    plus_prediction = image_to_predictions("plus_image.jpg")
    delta = plus_prediction-current_score
    print("Iteration "+str(i)+" delta:")
    print(delta)
    if (is_better(GOAL_COORDINATE, delta)):
        current_image[coordinate_numbering_rev[chosen_coordinate]]+=LEARNING_RATE 
    else:
        current_image[coordinate_numbering_rev[chosen_coordinate]]-=LEARNING_RATE

Iteration 0 score:
[9.98826861e-01 1.91611065e-07 3.88002022e-08 1.17289473e-03
 3.01193595e-08]
Iteration 0 delta:
[ 0.00000000e+00 -1.08002496e-12 -2.45137244e-13 -1.17579475e-08
 -2.41584530e-13]
Iteration 1 score:
[9.98713017e-01 1.95503304e-07 3.92091550e-08 1.28676998e-03
 2.96463138e-08]
Iteration 1 delta:
[-3.57627869e-06  8.19316881e-09  2.09722018e-09  3.44810542e-06
  3.67004205e-10]
Iteration 2 score:
[9.98793185e-01 2.09130476e-07 3.99210975e-08 1.20653852e-03
 2.99399758e-08]
Iteration 2 delta:
[ 1.99675560e-05 -6.06115691e-09 -4.04224210e-10 -2.00306531e-05
 -5.66162228e-10]
Iteration 3 score:
[9.98811483e-01 1.90833674e-07 3.63971715e-08 1.18828763e-03
 2.87970057e-08]
Iteration 3 delta:
[-4.97698784e-05  1.41352956e-08  4.89388086e-09  4.97497385e-05
  1.95761807e-09]
Iteration 4 score:
[9.98747885e-01 2.18077091e-07 3.71979247e-08 1.25180103e-03
 2.84209403e-08]
Iteration 4 delta:
[ 2.37822533e-05 -4.11004919e-09 -3.75379727e-11 -2.37227650e-05
 -3.85337984e-10]
Itera